In [1]:
# Load environment variables from a .env file
from dotenv import load_dotenv
import os

load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GOOGLE_API_KEY,
)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assistant that can classify the sentiment of text as Positive, Negative or Neutral."),
        ("user", "{user_input}")
    ]
)

In [5]:
chain = prompt | llm

In [9]:
response = chain.invoke({"user_input", "test is going great!"})
response.content

'Positive'

In [10]:
class State(dict):
    text: str
    sentiment: str
    response: str

In [15]:
from typing import Literal

# Nodes
def classify_sentiment(state: State) -> State:
    prompt = f"Is the following text Positive, Negative or Neutral in sentiment: {state['text']}"
    response = chain.invoke({"user_input": prompt})
    state["sentiment"] = response.content.strip().lower()
    return state

def positive_node(state: State) -> State:
    prompt = f"Generate an encouraging reply to this positive text:\n\n{state['text']}"
    response = chain.invoke({"user_input": prompt})
    state["response"] = response.content.strip()
    return state

def negative_node(state: State) -> State:
    prompt = f"Generate a supportive reply to this negative text:\n\n{state['text']}"
    response = chain.invoke({"user_input": prompt})
    state["response"] = response.content.strip()
    return state

def neutral_node(state: State) -> State:
    prompt = f"Generate a robotic reply to this neutral text:\n\n{state['text']}"
    response = chain.invoke({"user_input": prompt})
    state["response"] = response.content.strip()
    return state

def router_func(state: State) -> Literal["positive", "negative", "neutral"]:
    return "positive" if "positive" in state["sentiment"] else "negative" if "negative" in state["sentiment"] else "neutral"

In [16]:
from langgraph.graph import StateGraph, END

# Build the graph
workflow = StateGraph(State)

workflow.add_node("classification", classify_sentiment)
workflow.add_node("positive", positive_node)
workflow.add_node("negative", negative_node)
workflow.add_node("neutral", neutral_node)

# Classify node decides the flow
workflow.set_entry_point("classification")
workflow.add_conditional_edges("classification", router_func, {
    "positive": "positive",
    "negative": "negative",
    "neutral": "neutral"
})

# All branches from last step lead to END
workflow.add_edge("positive", END)
workflow.add_edge("negative", END)
workflow.add_edge("neutral", END)

# Compile graph
graph = workflow.compile()

In [21]:
# TEST
input_text = "I am happy about my progress today with this project."
result = graph.invoke({"text": input_text})

In [22]:
print("Sentiment:", result["sentiment"])
print("Response:", result["response"])

Sentiment: positive
Response: That's absolutely fantastic to hear! It's incredibly rewarding when you can see your hard work translate into tangible progress. Keep that momentum going – you're doing an amazing job!
